# TP1 Simulación

## Ejercicio 1

In [22]:
import numpy as np
import matplotlib.pyplot as plt
def own_rand():
    own_rand.x_n = (own_rand.x_n * 1013904223 + 1664525) % (2 ** 32)
    return own_rand.x_n
own_rand.x_n = (97811 + 97811 + 97811)/3


In [23]:
[own_rand() for i in range(6)]

[192755738, 1392371123, 1950649466, 726488403, 2540951770, 1889127667]

In [24]:
def own_rand_u():
    return own_rand()/(2**32)
